In [1]:
import math
import torch

class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        """
        LoRA(Low-Rank Adaptation) 레이어 초기화
        
        Args:
            in_dim: 입력 데이터의 차원 크기 (예: 768)
            out_dim: 출력 데이터의 차원 크기 (예: 768)
            rank: 병목 구간의 크기 (Rank, r). 작을수록 파라미터가 적어짐 (예: 8, 16)
            alpha: 스케일링 상수. 학습된 값의 영향력을 조절함
        """
        super().__init__()
        
        # 1. 행렬 A (Down-projection)
        # 입력(in_dim)을 작은 차원(rank)으로 압축하는 가중치입니다.
        # 파라미터로 등록하여 학습되도록 합니다.
        self.A = torch.nn.Parameter(torch.empty(in_dim, rank))
        
        # A는 무작위 값(Kaiming Uniform)으로 초기화합니다.
        # 이렇게 해야 다양한 특징을 학습할 준비가 됩니다.
        torch.nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
        
        # 2. 행렬 B (Up-projection)
        # 압축된 정보(rank)를 다시 원래 출력 차원(out_dim)으로 복원하는 가중치입니다.
        self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))
        
        # 중요: B는 '0'으로 초기화합니다.
        # 이유: 학습 시작 시점에는 A @ B의 결과가 0이 되어야 합니다.
        # 그래야 LoRA를 붙여도 기존 모델의 원래 출력값에 아무런 영향을 주지 않은 채로 시작할 수 있습니다.
        
        self.alpha = alpha
        self.rank = rank

    def forward(self, x):
        """
        순전파(Forward) 단계: 입력 x에 대한 LoRA 변화량(Delta)을 계산
        """
        # 1. 행렬 연산 (x @ A @ B)
        # 입력 x를 A와 곱해 차원을 줄이고(압축), 
        # 그 결과를 다시 B와 곱해 차원을 늘립니다(복원).
        # 연산 순서: (Batch, in) -> (Batch, rank) -> (Batch, out)
        
        # 2. 스케일링 (alpha / rank)
        # 학습된 결과에 상수배를 해줍니다. 
        # alpha는 강도 조절, rank로 나누는 것은 rank값이 바뀌어도 
        # 학습률(Learning Rate)을 크게 수정하지 않기 위한 정규화(Normalization) 과정입니다.
        x = (self.alpha / self.rank) * (x @ self.A @ self.B)
        
        return x

In [2]:
import torch

class LinearWithLoRA(torch.nn.Module):
    """
    기존의 Linear 레이어를 감싸서(Wrapping), 
    LoRA 어댑터를 추가한 새로운 복합 레이어를 만드는 클래스
    """
    def __init__(self, linear, rank, alpha):
        super().__init__()
        
        # 1. 기존 레이어 저장 (Frozen Weights)
        # 이미 학습된 레이어 가중치(W)를 그대로 가져옵니다.
        # 실제 사용 시에는 이 레이어의 가중치를 고정(freeze)시켜 학습되지 않게 합니다.
        self.linear = linear
        
        # 2. LoRA 레이어 생성 (Trainable Adapter)
        # 기존 레이어와 똑같은 입/출력 차원을 가지지만, 내부는 훨씬 가벼운 LoRA 레이어를 만듭니다.
        # linear.in_features, linear.out_features: 기존 레이어의 스펙을 그대로 베껴옵니다.
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        # 3. 결과 합치기 (The Core Logic)
        # 원래 모델이 하던 계산 결과 : self.linear(x)
        # LoRA가 새로 학습한 변화량 : self.lora(x)
        # 최종 결과 = 원래 결과 + 변화량
        return self.linear(x) + self.lora(x)

In [3]:
import torch

def replace_linear_with_lora(model, rank, alpha):
    """
    모델 내의 모든 Linear 레이어를 찾아 LoRA가 적용된 레이어로 교체하는 재귀 함수
    
    Args:
        model: 수정할 파이토치 모델 (또는 모델의 하위 모듈)
        rank: LoRA의 Rank (r)
        alpha: LoRA의 Alpha (스케일링 계수)
    """
    
    # 1. 자식 모듈 순회 (Iterate Children)
    # named_children(): 현재 모듈 바로 아래에 있는 구성 요소들을 (이름, 객체) 쌍으로 가져옵니다.
    # 예: ("attn", AttentionModule), ("mlp", MLPModule) ...
    for name, module in model.named_children():
        
        # 2. 교체 대상 확인 (Check Target)
        # 현재 보고 있는 모듈이 'torch.nn.Linear'인지 확인합니다.
        # (즉, 우리가 LoRA를 붙이고 싶은 기본 선형 층인지 확인)
        if isinstance(module, torch.nn.Linear):
            
            # 3. 레이어 교체 (Swap Layer)
            # setattr(객체, 속성이름, 새로운값): 객체의 속성을 동적으로 변경하는 파이썬 내장 함수
            # model의 'name'에 해당하는 레이어를 -> 'LinearWithLoRA'로 덮어씌웁니다.
            # 이때 기존 module을 인자로 넘겨줘서 가중치를 그대로 보존하게 합니다.
            setattr(model, name, LinearWithLoRA(module, rank, alpha))
            
        else:
            # 4. 재귀 호출 (Dig Deeper)
            # 만약 Linear 레이어가 아니라면(예: 트랜스포머 블록, 어텐션 층 등 컨테이너라면),
            # 그 내부에도 Linear 레이어가 숨어있을 수 있으므로
            # 그 안으로 들어가서 다시 똑같은 작업(replace_linear_with_lora)을 수행하라고 시킵니다.
            replace_linear_with_lora(module, rank, alpha)

In [4]:
from previous_chapters import load_gpt2_model
CHOOSE_MODEL = "gpt2-small (124M)"

BASE_CONFIG = {
    "vocab_size": 50257,     # 어휘사전 크기
    "context_length": 1024,  # 문맥 길이
    "drop_rate": 0.0,        # 드롭아웃 비율
    "qkv_bias": True         # 쿼리-키-값 편향
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12}, #트랜스포머 블록(Transformer Block)을 몇 층으로 쌓아 올렸는지
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model_name = "gpt2-small-124M.pth"
model = load_gpt2_model(model_name, BASE_CONFIG)
model.eval()

이미 파일이 존재합니다: models/gpt2\gpt2-small-124M.pth


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768,

In [5]:
# 1. 현재 학습 가능한(Trainable) 파라미터 개수 계산
# - model.parameters(): 모델 내부의 모든 가중치(W)와 편향(b)을 가져옵니다.
# - p.numel(): 각 파라미터 텐서의 전체 원소 개수(Number of Elements)를 셉니다.
# - if p.requires_grad: 현재 '학습 대상(True)'으로 설정된 것들만 필터링합니다.
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# {total_params:,} : 천 단위마다 쉼표(,)를 찍어서 보기 좋게 출력 (예: 1,000,000)
print(f"이전의 총 학습 가능 파라미터 수: {total_params:,}")


# 2. 모든 파라미터 얼리기 (Freezing)
# 모델의 모든 파라미터를 하나씩 꺼내서 반복합니다.
for param in model.parameters():
    # requires_grad = False:
    # "이 파라미터는 이제 학습시키지 마(기울기 계산 X)"라고 설정합니다.
    # 이렇게 하면 역전파(Backpropagation) 때 이 값들은 업데이트되지 않고 고정됩니다.
    param.requires_grad = False


# 3. 얼린 후 학습 가능 파라미터 수 재계산
# 위에서 모든 param의 requires_grad를 False로 바꿨으므로, 
# 조건(if p.requires_grad)을 만족하는 파라미터가 하나도 없게 됩니다.
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# 결과적으로 0이 출력되어야 정상입니다.
print(f"이후의 총 학습 가능 파라미터 수: {total_params:,}")

이전의 총 학습 가능 파라미터 수: 163,037,184
이후의 총 학습 가능 파라미터 수: 0


In [6]:
replace_linear_with_lora(model, rank=16, alpha=16)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"총 학습 가능 LoRA 파라미터 수: {total_params:,}")

총 학습 가능 LoRA 파라미터 수: 3,470,608
